Initialise the program

In [193]:
BingMaps_SubKey = ''
BingMaps_format = 'JPEG' #Options for PNG or GIF
BingMaps_Size = '600,600' # x,y or horizontal vs vertical
BingMaps_Style = 'Aerial' #Options include Aerial; AerialWithLabels; Road; see https://msdn.microsoft.com/en-us/library/ff701716.aspx
#Location_lat = ''
#Location_long = ''
Output_Map = "filename.jpg"
Read_CSV = "filename"
Output_CSV = Read_CSV + "-out"

Read_CSV += ".csv"
Output_CSV += ".csv"

import requests
import urllib.request
import json
import pandas as pd
from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt

Icon_DF = pd.DataFrame(data={'Company':["A", "B"], 'Large':[47,7], 'Medium':[44,8], 'Small':[14,9]})


Read CSV file from local drive and import into data file.

In [194]:
source_data = pd.read_csv(Read_CSV, sep=",", keep_default_na=False)
#print(source_data)
#print(source_data['City'])

Define a function to validate Latitude / Longitude coordinates

In [195]:
def CoordTest(num_in):
    if num_in:
        if not isinstance(num_in, float):
            num = float(num_in)
            if num > -180 and num < 180:
                num_bool = True
            else:
                num_bool = False
        else:
            if num > -180 and num < 180:
                num_bool = True
            else:
                num_bool = False
    else:
        num_bool = False
    return(num_bool)

Define a function to lookup using BING Maps API

http://dev.virtualearth.net/REST/v1/Locations/1%20Microsoft%20Way%20Redmond%20WA%2098052?o=xml&key=BingMapsKey

http://dev.virtualearth.net/REST/v1/Locations?countryRegion=countryRegion&adminDistrict=adminDistrict&locality=locality&postalCode=postalCode&addressLine=addressLine&userLocation=userLocation&userIp=userIp&usermapView=usermapView&includeNeighborhood=includeNeighborhood&maxResults=maxResults&key=BingMapsKey


Return the co-ordinates of the location

In [196]:
def BingLookup(Location):

    #Next line prepares a lookup string for inclusion in a URL - e.g. replaces spaces with %20
    encodedPostCode = urllib.parse.quote(Location, safe='')
    
    httprequest = 'http://dev.virtualearth.net/REST/v1/Locations/' + encodedPostCode + '?&key=' + BingMaps_SubKey
        
    request = urllib.request.Request(httprequest)
    response = urllib.request.urlopen(request)
    data = json.loads(response.read())
    
#    print(json.dumps(data, indent=8)) # used to print out the json response in a readable format to identify the nesting
    
    coordinates = data['resourceSets'][0]['resources'][0]['point']['coordinates']
        
    return(coordinates)

Define a function to provide Bing Map imagery of locations with pushpins

https://dev.virtualearth.net/REST/v1/Imagery/Map/imagerySet?pushpin={pushpin_1}&pushpin={pushpin_2}&pushpin={pushpin_n}&mapLayer={mapLayer}&format={format}&mapMetadata={mapMetadata}&key={BingMapsKey}  

Define a function to use the address to lookup the postcode if a postcode doesn't exist

In [197]:
def BingPostcodeLookup(Address, City, row):
    Lookup_Address = Address + " , " + City
    
    #Next line prepares a lookup string for inclusion in a URL - e.g. replaces spaces with %20
    encodedLookup_Address = urllib.parse.quote(Lookup_Address, safe='')
    
    httprequest = 'http://dev.virtualearth.net/REST/v1/Locations/' + encodedLookup_Address + '?&key=' + BingMaps_SubKey
        
    request = urllib.request.Request(httprequest)
    response = urllib.request.urlopen(request)
    data = json.loads(response.read())
    
#    print(json.dumps(data, indent=8))
    
    if 'postalCode' not in data['resourceSets'][0]['resources'][0]['address']:
        postcode = "Not Found"
    else:
        postcode = data['resourceSets'][0]['resources'][0]['address']['postalCode']
    
#    print(postcode)
    
    new_postcode = pd.DataFrame({'Postcode': [postcode]}, index=[row])
    source_data.update(new_postcode)
    
#    return(coordinates)    


Function to work out and update the pushpin icons

In [198]:
def Pushpin_Icon(Company, Users, Row):
    Count_Icon_DF = 0
    new_Icon = 10
    for C in Icon_DF["Company"]:
        if C == Company:
            if Users <300:
                new_Icon = Icon_DF["Medium"][Count_Icon_DF]
                if Users <100:
                    new_Icon = Icon_DF["Small"][Count_Icon_DF]
            else:
                new_Icon = Icon_DF["Large"][Count_Icon_DF]
    
                
        Count_Icon_DF +=1
    
    new_Icon_update = pd.DataFrame({'Icon': [new_Icon]}, index=[Row])
    source_data.update(new_Icon_update)

Function to produce the map and save locally.

In [199]:
def BingMapImage(pushpin_location):
    
    Map_Lookup = 'https://dev.virtualearth.net/REST/v1/Imagery/Map/' + BingMaps_Style + '?' + pushpin_location + "mapSize=" + BingMaps_Size + "&format=" + BingMaps_format + "&key=" + BingMaps_SubKey

    Map_request = urllib.request.urlretrieve(Map_Lookup, Output_Map)

    # following code to make the map appear within the Jupyter Notebook
#    image=Image.open(Output_Map)
    
#    plt.imshow(image, aspect='auto')
#    plt.show()

    
    #print(Map_Lookup) #used so you can manually open the HTTPS request.

Define a function to update datafile with the found co-ordinates

In [200]:
def UpdateDF(Location_lat, Location_long):

    new_lat = pd.DataFrame({'Latitude': [Location_lat]}, index=[row_count])
    new_long = pd.DataFrame({'Longitude': [Location_long]}, index=[row_count])
    source_data.update(new_lat)
    source_data.update(new_long)    


Define a function to create new CSV file with updated data

In [201]:
def UpdateCSV():
    source_data.to_csv(Output_CSV, encoding='utf-8', index=False)

Main part of the program to cycle through each row and call BingLookup to pickup co-ordinates based on Postcode or City.

Update CSV with found co-ordinates.

Once all locations have been identified, call BingMapImage to create a map based on the locations.

In [202]:
row_count = 0
pushpin_location = ""
for Location in source_data["Reference"]:
    if source_data["Postcode"][row_count]:
        Location = source_data["Postcode"][row_count]
    else:
        BingPostcodeLookup(source_data["Address"][row_count],source_data["City"][row_count],row_count)

    Location_lat = source_data["Latitude"][row_count]
    Location_long = source_data["Longitude"][row_count]

    LatTest_Return = CoordTest(Location_lat)
    LongTest_Return = CoordTest(Location_long)

    if not LatTest_Return and not LongTest_Return:
        coordinates = BingLookup(Location)
        Location_lat = coordinates[0]
        Location_long = coordinates[1]
        UpdateDF(Location_lat, Location_long) #update Dataframe with found location long/lat
    
    #Site reference section
    Site_Ref = source_data["Reference"][row_count]
    if len(Site_Ref) > 3:
        Site_Ref_Short = Site_Ref[:3]
    else:
        Site_Ref_Short = Site_Ref
   
    #Site Icon section
    Site_Icon = source_data["Icon"][row_count]
    if not Site_Icon:
        Pushpin_Icon(source_data["Company"][row_count], source_data["Users"][row_count], row_count)
        Site_Icon1 = source_data["Icon"][row_count]
    
    Site_Icon = int(round(Site_Icon1))
    
    #Create pushpin
    if not source_data["Postcode"][row_count] == "Not Found":        
        pushpin_location +="pushpin=" + str(Location_lat) + "," + str(Location_long) + ";" + str(Site_Icon) + ";" + Site_Ref_Short + '&'
    
    row_count +=1
    
BingMapImage(pushpin_location) #create Map based on locations using Bing Maps

#print(source_data)
UpdateCSV() #create a new file with location latitude / longitude included.

#print("End")